In [1]:
# import pandas as pd

# # List of CSV files to merge
# csv_files = ['dares/test.csv', 'dares/train.csv', 'dares/validation.csv']

# # Read and concatenate all CSV files
# merged_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# # Save the merged dataframe to a new CSV file
# merged_df.to_csv('merged_output.csv', index=False)

# print("CSV files merged successfully into 'merged_output.csv'")

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("merged_output.csv",sep="\t")

In [4]:
df_math = df[df["English_Filename"]=="Math"].reset_index(drop=True)
df_Science = df[df["English_Filename"]=="Science"].reset_index(drop=True)
df_Physics = df[df["English_Filename"]=="Physics"].reset_index(drop=True)
df_Ecology = df[df["English_Filename"]=="Ecology"].reset_index(drop=True)
df_Chemistry = df[df["English_Filename"]=="Chemistry"].reset_index(drop=True)
df_Biology = df[df["English_Filename"]=="Biology"].reset_index(drop=True)

In [5]:
df_Arabic1 = df[df["English_Filename"]=="ArabicLanguage"].reset_index(drop=True)
df_Arabic2 = df[df["English_Filename"]=="Arabic_Language"].reset_index(drop=True)

In [6]:
df_Arabic = pd.concat([df_Arabic1,df_Arabic2], ignore_index=True)

In [7]:
# import faiss
# import numpy as np
# from FlagEmbedding import BGEM3FlagModel
# import torch
# from tqdm.notebook import tqdm
# import pickle

# # Ensure GPU is used if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if device.type != "cuda":
#     raise RuntimeError("CUDA device not available. Ensure GPU and faiss-gpu are properly installed.")

# # Initialize the BGE-M3 model
# model = BGEM3FlagModel('BAAI/bge-m3', device=device)

# # Initialize FAISS GPU resources and index
# dimension = 1024  # BGE-M3 embedding dimension
# res = faiss.StandardGpuResources()  # Create GPU resources
# index = faiss.IndexFlatL2(dimension)  # Create CPU index
# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # Move index to GPU (device 0)

# # List to store metadata
# metadata = []

# # Function to process and add embeddings for a given dataframe
# def add_embeddings_to_index(df, subject_name, index, metadata_list):
#     embeddings = []
#     for i in tqdm(range(len(df)), desc=f"Encoding {subject_name}"):
#         emb = model.encode(df["Text"][i], max_length=256)['dense_vecs']
#         embeddings.append(emb)
        
#         # Store only text and subject as metadata
#         meta = {'subject': subject_name, 'text': df["Text"][i]}
#         metadata_list.append(meta)
    
#     # Convert embeddings to numpy array and add to FAISS GPU index
#     embeddings = np.array(embeddings).astype('float32')
#     index.add(embeddings)
#     return embeddings

# # Process each subject's dataframe
# subjects = [
#     ('Math', df_math),
#     ('Arabic', df_Arabic),
#     ('Science', df_Science),
#     ('Physics', df_Physics),
#     ('Ecology', df_Ecology),
#     ('Chemistry', df_Chemistry),
#     ('Biology', df_Biology)
# ]

# for subject_name, df in subjects:
#     add_embeddings_to_index(df, subject_name, gpu_index, metadata)

# # Move index back to CPU for saving (optional, if you need CPU compatibility)
# cpu_index = faiss.index_gpu_to_cpu(gpu_index)
# faiss.write_index(cpu_index, "subject_embeddings.faiss")

# # Save metadata
# with open("subject_metadata.pkl", "wb") as f:
#     pickle.dump(metadata, f)

# print(f"FAISS GPU index created with {gpu_index.ntotal} vectors.")

In [8]:
# import faiss
# import numpy as np
# from FlagEmbedding import BGEM3FlagModel
# import torch
# import pickle
# from tqdm import tqdm

# # Ensure GPU is used if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if device.type != "cuda":
#     raise RuntimeError("CUDA device not available. Ensure GPU and faiss-gpu are properly installed.")

# # Initialize the BGE-M3 model
# model = BGEM3FlagModel('BAAI/bge-m3', device=device)

# # Load the FAISS index
# cpu_index = faiss.read_index("subject_embeddings.faiss")

# # Move index to GPU
# res = faiss.StandardGpuResources()
# gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

# # Load metadata
# with open("subject_metadata.pkl", "rb") as f:
#     metadata = pickle.load(f)

# # Function to query the FAISS index with subject filter
# def search_faiss_with_subject_filter(query_text, index, model, metadata, subject_filter=None, top_k=5):
#     # Encode the query text
#     query_embedding = model.encode(query_text, max_length=256)['dense_vecs']
#     query_embedding = np.array([query_embedding]).astype('float32')  # Shape: (1, dimension)

#     # Perform similarity search (retrieve more results to account for filtering)
#     search_k = top_k * 10  # Oversample to ensure enough results after filtering
#     distances, indices = index.search(query_embedding, search_k)

#     # Filter results by subject and collect top-k
#     results = []
#     for i, idx in enumerate(indices[0]):
#         if idx >= len(metadata):  # Safety check for index bounds
#             continue
#         meta = metadata[idx]
#         # Apply subject filter (case-insensitive)
#         if subject_filter is None or meta['subject'].lower() == subject_filter.lower():
#             result = {
#                 'index': int(idx),
#                 'distance': float(distances[0][i]),
#                 'subject': meta['subject'],
#                 'text': meta['text']
#             }
#             results.append(result)
#         if len(results) >= top_k:  # Stop once we have enough filtered results
#             break
    
#     return results

# # Example query with subject filter
# query = "ما هي الخليه"
# subject_filter = None  # Set to None for no filtering, or specify a subject like "Math"
# top_k = 5  # Number of top results to retrieve
# results = search_faiss_with_subject_filter(query, gpu_index, model, metadata, subject_filter, top_k)

# # Print results
# print(f"Query: {query}")
# print(f"Subject Filter: {subject_filter if subject_filter else 'None'}")
# print(f"Top {len(results)} results (up to {top_k}):")
# for i, result in enumerate(results, 1):
#     print(f"\nResult {i}:")
#     print(f"Subject: {result['subject']}")
#     print(f"Text: {result['text']}")
#     print(f"Distance: {result['distance']:.4f}")

In [ ]:
import faiss
import numpy as np
from FlagEmbedding import BGEM3FlagModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Dict
import pickle

# Ensure GPU is used if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type != "cuda":
    print("Warning: CUDA not available. Using CPU, which may be slower.")

# Initialize the BGE-M3 model for embeddings
embedding_model = BGEM3FlagModel('BAAI/bge-m3', device=device)

# Load the FAISS index
cpu_index = faiss.read_index("subject_embeddings.faiss")

# Move index to GPU if available
if device.type == "cuda":
    res = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
else:
    gpu_index = cpu_index  # Fallback to CPU index

# Load metadata
with open("subject_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

# Initialize the Qwen3-0.6B model and tokenizer
llm_model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    torch_dtype="auto",
    device_map="auto"
)


# Function to query FAISS without subject filter and no oversampling
def search_faiss(query_text: str, index, embedding_model, metadata: List[Dict], top_k: int = 3) -> List[Dict]:
    query_embedding = embedding_model.encode(query_text, max_length=256)['dense_vecs']
    query_embedding = np.array([query_embedding]).astype('float32')

    search_k = top_k
    distances, indices = index.search(query_embedding, search_k)

    results = []
    for i, idx in enumerate(indices[0]):
        if idx >= len(metadata):
            continue
        result = {
            'index': int(idx),
            'distance': float(distances[0][i]),
            'subject': metadata[idx]['subject'],
            'text': metadata[idx]['text']
        }
        results.append(result)
        if len(results) >= top_k:
            break
    
    return results

# Function to call the Qwen3-0.6B model with chat template
def call_llm(prompt: str, history: List[Dict[str, str]] = [], max_new_tokens: int = 200) -> str:
    # Prepare chat history
    messages = []
    for turn in history[-3:]:
        messages.append({"role": "user", "content": turn['user']})
        messages.append({"role": "assistant", "content": turn['assistant']})
    messages.append({"role": "user", "content": prompt})

    # Apply chat template with enable_thinking=False
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(llm_model.device)

    # Generate response
    generated_ids = llm_model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # Decode the response
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
    return content

# Conversational RAG function
def conversational_rag(query: str, history: List[Dict[str, str]]) -> Dict[str, str]:
    results = search_faiss(query, gpu_index, embedding_model, metadata, top_k=3)
    context = "\n".join([f"- {result['text']} (موضوع: {result['subject']})" for result in results])
    
    prompt = f"""
أنا معلم ودود أشرح المواد المدرسية للطلاب من الصف الأول إلى الصف الثاني عشر. سأقدم الإجابات باللغة العربية فقط بأسلوب بسيط وودود يناسب مستواكم. لا أستخدم أي لغة أخرى.
السياق (استخدمه إذا كان مفيدًا):
{context}

السؤال: {query}
أجب بالعربية الفصحى بطريقة ودودة وسهلة الفهم تناسب الطلاب من جميع المراحل الدراسية، وإذا لم يكن السياق كافيًا، استخدم معرفتي لتوضيح الموضوع بشكل ممتع!
"""
    
    response = call_llm(prompt, history)
    history.append({"user": query, "assistant": response})
    
    return {"content": response, "history": history}

# Example conversational loop
def main():
    history = []
    print("مرحبًا يا أصدقاء! أنا معلم ودود هنا لمساعدتكم في الدراسة من الصف الأول إلى الصف الثاني عشر. اسألوني سؤالًا بالعربية (اكتب 'خروج' للخروج):")
    while True:
        query = input("السؤال: ")
        if query.strip() == "خروج":
            break
        result = conversational_rag(query, history)
        print(f"الإجابة: {result['content']}\n")
        history = result['history'][-10:]

if __name__ == "__main__":
    main()

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

مرحبًا يا أصدقاء! أنا معلم ودود هنا لمساعدتكم في الدراسة من الصف الأول إلى الصف الثاني عشر. اسألوني سؤالًا بالعربية (اكتب 'خروج' للخروج):


السؤال:  ما هي القوانين في الفيزياء؟


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


الإجابة: القوام في الفيزياء هي قوانين تساعد في فهم الظواهر الطبيعية. ومنها:

- القوانين الأولية في الفيزياء، مثل قوانين نيوتن، تُوضح أن الحركة تظل كما هي إذا لم تكن القوة المحصلة على الجسم متساوية بالصفر.
- القوانين الأخرى مثل القوانين الفيزيائية لنيوتن، والتي توضّحت أن القوى تتأثر بشكل معاكس بينها، كما تساعد في تنبؤات مثل تجربة نيوتن والمحركات.



السؤال:  ما هي الخليه؟


الإجابة: الخليه هي الوحدة الأساسية لبناء الجسم الحي، وتكون من أجزاء الجسم. تتكون الخلايا من مواد مختلفة تؤدي وظائف محددة. الخلايا الحية تحتوي على عدة أجزاء داخلية، مثل النواة، السيتوبلازما، وغشاء الخلوي. تساعد الخلايا في تنظيم وظائف الجسم، مثل توزيع الوراثة والتكاثر. كل خلاصة تلعب دورًا مركبًا في الحفاظ على صحة الكائن الحي.

